In [2]:
import pandas as pd
import os 
import glob
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import datetime 

# Define paths
folder_path = '/Users/jud-05/Desktop/High Court/ANNUAL'
new_path = '/Users/jud-05/Desktop/High Court/ANNUAL_COMBINED'
path = '/Users/jud-05/Desktop/High Court/ANNUAL_COMBINED/dcrt_4Q'

# Ensure the new directories exist
os.makedirs(new_path, exist_ok=True)
os.makedirs(path, exist_ok=True)

# Reading all the Excel files in the folder
file_names = glob.glob(folder_path + "/*.xls")

# Iterate through each file
for f in file_names:
    # Read Excel file into a DataFrame, skipping the first 4 rows
    df = pd.read_excel(f, skiprows=4)
    
    # Extract the base name of the file without extension and with a unique identifier
    base_name = os.path.splitext(os.path.basename(f))[0]  # Get base name without extension
    base_name = base_name.split()[0] + '_' + str(file_names.index(f))  # Append unique index
    
    # Save it as a .csv in the new directory
    df.to_csv(f'{path}/{base_name}.csv', index=False)


In [3]:
# Use glob to list all CSV files in the folder
csv_files = glob.glob(path + "/*.csv")
# Count the number of CSV files
num_csv_files = len(csv_files)

print(f"Number of CSV files in the folder: {num_csv_files}")

Number of CSV files in the folder: 312


In [4]:
# Get all the CSV files in the directory
all_files = glob.glob(path + "/*.csv")

all_files

# Combine all the CSV files into a single DataFrame and insert a column with the name of the file
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['file_name'] = os.path.basename(filename)
    li.append(df)
       
combined_csv = pd.concat(li, axis=0, ignore_index=True)


#save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_32255/2607724255.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_csv = pd.concat(li, axis=0, ignore_index=True)


In [5]:
#rename columns
# Simplified version of data.rename() function
combined_csv.rename(columns={
    'file_name': 'court_name',
    'Day': 'date_dd',
    'Month': 'date_mon',
    'Year': 'date_yyyy',
    'Code': 'caseid_type',
    'No.': 'caseid_no',
    'Day.1': 'filed_dd',
    'Month.1': 'filed_mon',
    'Year.1': 'filed_yyyy',
    'Name of Court': 'origcourt',
    'Code.1': 'appeal_type',
    'No..1': 'appeal_no',
    'Year.2': 'appeal_yyyy',
    '\n(Select)': 'case_type',
    'Name of Judge 1 or Magistrate/DR or Kadhi': 'bench_1',
    'Judge 2': 'bench_2',
    'Judge 3': 'bench_3',
    'Judge 4': 'bench_4',
    'Judge 5': 'bench_5',
    'Judge 6': 'bench_6',
    'Judge 7': 'bench_7',
    '(Select option)': 'comingfor',
    '(Select option).1': 'outcome',
    '(Select option).2': 'reason_adj',
    'Day.2': 'next_dd',
    'Month.2': 'next_mon',
    'Year.3': 'next_yyyy',
    'M': 'firstparty_m',
    'F': 'firstparty_f',
    'Org.': 'firstparty_org',
    'M.1': 'secondparty_m',
    'F.1': 'secondparty_f',
    'Org..1': 'secondparty_o',
    '(Y/N)': 'legalrep',
    'P \n(Enter No.)': 'p_witness',
    'D\n(Enter No.)': 'd_witness',
    '(Enter No.)': 'custody',
    '(Free text)': 'other_details'
}, inplace=True)

#save the combined file as a csv
combined_csv.to_csv(f'{path}/raw data/HC_combined.csv', index=False)

In [6]:
def convert_to_datetime(combined_csv, day_col, month_col, year_col, new_col, index):
    date_str = combined_csv[day_col].astype(str) + "-" + combined_csv[month_col] + "-" + combined_csv[year_col].astype(str)
    combined_csv[new_col] = pd.to_datetime(date_str, format="%d-%b-%Y", errors="coerce")
    #data.insert(index, new_col, data.pop(new_col))
    return combined_csv

combined_csv = convert_to_datetime(combined_csv, "date_dd", "date_mon", "date_yyyy", "activity_date", 2)
combined_csv = convert_to_datetime(combined_csv, "filed_dd", "filed_mon", "filed_yyyy", "filed_date", 3)
for i in combined_csv['next_dd'].unique():
    if i == " ":
        combined_csv.loc[combined_csv[['next_dd', 'next_mon', 'next_yyyy']] == i, 'next_dd', 'next_mon', 'next_yyyy'] = ""
    else:
        combined_csv = convert_to_datetime(combined_csv, "next_dd", "next_mon", "next_yyyy", "next_date", 4)
        
combined_csv['activity_date'].value_counts()

activity_date
2023-09-28    3163
2023-07-27    3160
2023-10-12    3019
2023-11-30    2976
2024-03-18    2938
              ... 
2023-07-02       1
2023-12-10       1
2024-06-02       1
2023-07-30       1
2023-09-10       1
Name: count, Length: 310, dtype: int64

In [6]:
#create unique case id
combined_csv["case_identifier"] = combined_csv ['court_name'] + "-" + combined_csv["caseid_type"]+ "-" + combined_csv["caseid_no"].astype(str) + "/" + combined_csv["filed_date"].astype(str)
combined_csv.insert(1, "case_identifier", combined_csv.pop("case_identifier"))

combined_csv["case_identifier"].value_counts()

case_identifier
Mombasa_36.csv-HCCRC-E017/2023-08-29         11
Homabay_57.csv-HCCRC-E002/2024-01-17          7
Milimani_6.csv-HCCC-1068/1998-05-07           6
Milimani_40.csv-HCFP&A-594/2002-07-13         6
Milimani_28.csv-HCCHRPET-E181/2024-04-08      6
                                             ..
Milimani_5.csv-HCCOMMMISC-E036/2024-01-18     1
Milimani_5.csv-HCCOMMMISC-E038/2024-01-19     1
Milimani_5.csv-HCCOMMMISC-E040/2024-01-19     1
Milimani_5.csv-HCCOMMMISC-E050/2024-01-24     1
Maralal_42.csv-HCCRREV-E001/2024-02-15        1
Name: count, Length: 46080, dtype: int64

In [7]:
#check duplicates
combined_csv.duplicated().sum()

#drop duplicates
combined_csv = combined_csv.drop_duplicates()



In [8]:
combined_csv['court_name'].value_counts()

court_name
Milimani_5.csv     5620
Milimani_40.csv    5579
Milimani_6.csv     4106
Kiambu_76.csv      2273
Machakos_56.csv    1792
                   ... 
Eldama_39.csv        90
Marsabit_71.csv      77
Eldoret_37.csv       55
Maralal_42.csv       31
Mandera_64.csv       18
Name: count, Length: 76, dtype: int64

In [9]:
combined_csv['case_type'].value_counts()

case_type
Civil Appeal                                                     8899
Family P&A Intestate                                             6782
Murder Case                                                      5387
Civil Case Miscellaneous                                         4619
Criminal Appeal                                                  4048
Commercial Matters                                               3266
Criminal Revision                                                2919
Civil Suit                                                       2494
Constitution and Human Rights Petition (Civil)                   1848
Criminal Miscellaneous Application                               1825
Commercial Miscellaneous                                         1377
Family P&A Testate                                               1094
Family P&A Citation                                              1089
Family P&A Ad Litem                                               985
Family Mis

In [10]:
def create_case_nature(combined_csv, old_col, new_col, index):
    case_nature = combined_csv[old_col].astype(str).apply(lambda x: 
        "CRIMINAL" if any(y in x for y in ["CR"]) 
        else "CIVIL")
    combined_csv[new_col] = case_nature
   #combined_csv.insert(index, new_col, combined_csv.pop(new_col))
    return combined_csv
combined_csv = create_case_nature(combined_csv, "caseid_type", "case_nature", 5)




In [11]:
combined_csv['outcome'].value_counts()

outcome
Mention date set (in court)     2753
Case Registered/Filed           2011
Directions given                1845
Hearing date set (in court)      748
Adjournment                      623
                                ... 
Ruling Date Set                    1
Mention date at the Registry       1
Oral submissions made              1
Limited Grant Issued               1
Grant revoked                      1
Name: count, Length: 73, dtype: int64

In [12]:
#generate function to generate filed cases width activity date before filed date
def filed_cases(combined_csv):
    filed = []
    for i, row in combined_csv.iterrows():
        if "Case Registered/Filed" in str(row['outcome']) and row['activity_date'] >= row['filed_date']:
            filed.append(1)
        else:
            filed.append(0)
    combined_csv['filed'] = filed
    return combined_csv

combined_csv = filed_cases(combined_csv)

combined_csv['filed'].value_counts()

filed
0    51866
1     1978
Name: count, dtype: int64

In [13]:
combined_csv['outcome'].value_counts()

outcome
Mention date set (in court)     2753
Case Registered/Filed           2011
Directions given                1845
Hearing date set (in court)      748
Adjournment                      623
                                ... 
Ruling Date Set                    1
Mention date at the Registry       1
Oral submissions made              1
Limited Grant Issued               1
Grant revoked                      1
Name: count, Length: 73, dtype: int64

In [14]:
#change outcome column to upper case
combined_csv['outcome'] = combined_csv['outcome'].str.upper()
def resolved_cases(combined_csv):
    resolved = []
    for i in combined_csv['outcome'].astype(str):
        if ("ABATED" in i or "CASE CLOSED" in i or "STRUCK OUT" in i or  "TERMINATED" in i or "DISMISS" in i or "GRANT CONFIRMED" in i or "GRANT REVOKED" in i or "JUDGMENT DELIVERED" in i or "ACQUITTAL" in i or "CONVICTED" in i or "LIMITED GRANT ISSUED" in i or "WITHDRAW" in i or "SETTLEMEN" in i or "REVISION DECLINED" in i or "RULING DELIVERED- ACCUSED DISC" in i or "RULING DELIVERED- CASE CLOSED" in i) and ("ORDER ISSUED - CASE CLOSED" not in i):
            resolved.append(1)
    
        else:
            resolved.append(0)
    combined_csv['resolved'] = resolved
    return combined_csv

combined_csv = resolved_cases(combined_csv)

total_resolved = combined_csv['resolved'].sum()
combined_csv[['outcome', 'resolved']] [combined_csv['resolved'] == 1].value_counts()
total_resolved






1055

In [15]:
#resolved by court name
resolved_court = combined_csv.groupby(['court_name'])['resolved'].sum().reset_index()
resolved_court['resolved'] = resolved_court['resolved'].astype(int)

resolved_court = resolved_court.sort_values(by=['court_name'], ascending=True)

resolved_data = combined_csv[combined_csv['resolved'] == 1]

resolved_data['court_name'].value_counts()

court_name
Kakamega_44.csv    100
Milimani_40.csv     87
Nanyuki_14.csv      76
Milimani_5.csv      70
Malindi_65.csv      67
                  ... 
Nyamira_67.csv       1
Milimani_77.csv      1
Nyamira_9.csv        1
Nakuru_23.csv        1
Mombasa_36.csv       1
Name: count, Length: 64, dtype: int64

In [16]:
# generate age of resolved cases

def time_to_resolve(resolved_data):
    time_to_resolve = []
    for i in resolved_data['court_name']:
        time_to_resolve.append(i)
    resolved_data.loc[:, 'time_to_resolve'] = resolved_data['activity_date'] - resolved_data['filed_date']
    return resolved_data

resolved_data = time_to_resolve(resolved_data)

resolved_data['time_to_resolve'] = resolved_data['time_to_resolve'].dt.days
resolved_data['time_to_resolve'] = resolved_data['time_to_resolve'].fillna(0).astype(int)

#drop if time to resolve is less than 0
resolved_data = resolved_data[resolved_data['time_to_resolve'] >= 0]

resolved_data['time_to_resolve'].value_counts()

/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_30001/2198342833.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_data.loc[:, 'time_to_resolve'] = resolved_data['activity_date'] - resolved_data['filed_date']
/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_30001/2198342833.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_data['time_to_resolve'] = resolved_data['time_to_resolve'].dt.days
/var/folders/5q/4jwyrq8j3_95g9th989sqyfw0000gn/T/ipykernel_30001/2198342833.

time_to_resolve
31      44
9       34
30      32
22      30
23      26
        ..
1036     1
699      1
1818     1
978      1
1548     1
Name: count, Length: 568, dtype: int64

In [17]:
#combine  'bench_1', 'bench_2', 'bench_3', 'bench_4', 'bench_5', 'bench_6','bench_7 as a single column
resolved_data['bench'] = resolved_data[['bench_1', 'bench_2', 'bench_3', 'bench_4', 'bench_5', 'bench_6', 'bench_7']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)


In [18]:
resolved_data['bench'].value_counts()

bench
Otieno, Patrick              101
Ndung'u, Anthony              98
Githinji, Stephen             64
Ogola, Daniel Ogembo          52
Gitari, Lucy                  32
                            ... 
Karani, Elizabeth Wairimu      1
Waswa, Cyprian  Wafula         1
 Mati, Martin Muya             1
 Mulwa, Janet                  1
Menya, Christine Achieng       1
Name: count, Length: 103, dtype: int64

In [19]:
#generate mean time to resolve for each bench by case nature
bench_mean_time = resolved_data.groupby(['bench', 'case_nature'])['time_to_resolve'].mean().reset_index()
bench_mean_time['time_to_resolve'] = bench_mean_time['time_to_resolve'].astype(int)
bench_mean_time = bench_mean_time.sort_values(by=['time_to_resolve'], ascending=False)
bench_mean_time



,bench,case_nature,time_to_resolve
125,"Wamae, Thripsisa Cherere",CIVIL,9467
8,"Mati, Martin Muya",CIVIL,6512
92,"Odero, Maureen Akinyi",CIVIL,6505
106,"Otieno, Patrick",CIVIL,6344
51,"Masiga, James Jesse",CIVIL,6334
...,...,...,...
120,"Thande, Mugure",CRIMINAL,13
78,"Njuguna, Lucy Mwihaki",CIVIL,12
67,"Mutuku, Stella Ngali",CIVIL,12
124,"Wakiaga, James",CRIMINAL,10


In [20]:
'''def merit_resolution(resolved_data):
    merit = []
    other = []
    for i in resolved_data['outcome']:
        if "JUDGMENT DELIVERED" in i or "RULING DELIVERED- CASE CLOSED" in i:
            merit.append(1)
            other.append(0)
        else:
            merit.append(0)
            other.append(1)
    resolved_data['merit_resolution'] = merit
    resolved_data['other_resolution'] = other
    return resolved_data

resolved_data = merit_resolution(resolved_data)'''


'def merit_resolution(resolved_data):\n    merit = []\n    other = []\n    for i in resolved_data[\'outcome\']:\n        if "JUDGMENT DELIVERED" in i or "RULING DELIVERED- CASE CLOSED" in i:\n            merit.append(1)\n            other.append(0)\n        else:\n            merit.append(0)\n            other.append(1)\n    resolved_data[\'merit_resolution\'] = merit\n    resolved_data[\'other_resolution\'] = other\n    return resolved_data\n\nresolved_data = merit_resolution(resolved_data)'

In [21]:
def merit_resolution(resolved_data):
    resolution_nature = []
    for i in resolved_data['outcome']:
        if "JUDGMENT DELIVERED" in i or "RULING DELIVERED- CASE CLOSED" in i:
            resolution_nature.append('merit')
        else:
            resolution_nature.append('other')
    resolved_data['resolution_nature'] = resolution_nature
    return resolved_data

resolved_data = merit_resolution(resolved_data)

In [22]:
resolved_data['resolution_nature'].value_counts()

resolution_nature
other    669
merit    385
Name: count, dtype: int64

In [23]:
#function to create a column for resolution case nature using resolution nature and case nature
def resolution_case_nature(resolved_data):
    resolution_case_nature = []
    for i, row in resolved_data.iterrows():
        if row['resolution_nature'] == 'merit' and row['case_nature'] == 'CRIMINAL':
            resolution_case_nature.append('merit_criminal')
        elif row['resolution_nature'] == 'merit' and row['case_nature'] == 'CIVIL':
            resolution_case_nature.append('merit_civil')
        elif row['resolution_nature'] == 'other' and row['case_nature'] == 'CRIMINAL':
            resolution_case_nature.append('other_criminal')
        else:
            row['resolution_nature'] == 'other' and row['case_nature'] == 'CIVIL'
            resolution_case_nature.append('other_civil')
    resolved_data['resolution_case_nature'] = resolution_case_nature
    return resolved_data

resolved_data = resolution_case_nature(resolved_data)

resolved_data['resolution_case_nature'].value_counts()

resolution_case_nature
other_civil       424
other_criminal    245
merit_criminal    231
merit_civil       154
Name: count, dtype: int64

In [24]:
#resolution case nature by bench
resolution_bench = resolved_data.groupby(['bench', 'resolution_case_nature'])['case_identifier'].count().reset_index()
resolution_bench = resolution_bench.sort_values(by=['case_identifier'], ascending=False)
resolution_bench

#pivot table for resolution case nature by bench
resolution_bench_pivot = resolved_data.pivot_table(index='bench', columns='resolution_case_nature', values='case_identifier', aggfunc='count').reset_index()
resolution_bench_pivot = resolution_bench_pivot.fillna(0)
resolution_bench_pivot = resolution_bench_pivot.sort_values(by=['merit_civil', 'merit_criminal', 'other_civil', 'other_criminal'], ascending=False)
resolution_bench_pivot

#change to integer
resolution_bench_pivot['merit_civil'] = resolution_bench_pivot['merit_civil'].astype(int)
resolution_bench_pivot['merit_criminal'] = resolution_bench_pivot['merit_criminal'].astype(int)
resolution_bench_pivot['other_civil'] = resolution_bench_pivot['other_civil'].astype(int)
resolution_bench_pivot['other_criminal'] = resolution_bench_pivot['other_criminal'].astype(int)
resolution_bench_pivot

#rename columns
resolution_bench_pivot.columns = ['Bench', 'Merit Civil', 'Merit Criminal', 'Others Civil', 'Other Criminal']
resolution_bench_pivot

,Bench,Merit Civil,Merit Criminal,Others Civil,Other Criminal
80,"Ongeri, Asenath",19,0,7,0
17,"Chigiti,john Mugwimi",13,0,3,0
14,"Chembeni, Louser Adisa",11,0,9,0
64,"Noelyne, Reuben Akee",10,0,6,0
22,"Githogori, Stephany W.",9,0,4,0
...,...,...,...,...,...
49,"Musya, Musyoka William",0,0,0,10
26,"Karanja, Joseph R.",0,0,0,3
32,"Kimondo, Kanyi",0,0,0,3
52,"Mutende, Lilian Nambwire",0,0,0,3


In [25]:
#generate mean time to resolve for each bench by resolution case nature
bench_mean_time_resolution = resolved_data.groupby(['bench', 'resolution_case_nature'])['time_to_resolve'].mean().reset_index()
bench_mean_time_resolution['time_to_resolve'] = bench_mean_time_resolution['time_to_resolve'].astype(int)
bench_mean_time_resolution = bench_mean_time_resolution.sort_values(by=['time_to_resolve'], ascending=False)
bench_mean_time_resolution

#pivot table for mean time to resolve by bench and resolution case nature
bench_mean_time_resolution_pivot = resolved_data.pivot_table(index='bench', columns='resolution_case_nature', values='time_to_resolve', aggfunc='mean').reset_index()
bench_mean_time_resolution_pivot = bench_mean_time_resolution_pivot.fillna(0)
bench_mean_time_resolution_pivot = bench_mean_time_resolution_pivot.sort_values(by=['merit_civil', 'merit_criminal', 'other_civil', 'other_criminal'], ascending=False)
bench_mean_time_resolution_pivot

#change to integer
bench_mean_time_resolution_pivot['merit_civil'] = bench_mean_time_resolution_pivot['merit_civil'].astype(int)
bench_mean_time_resolution_pivot['merit_criminal'] = bench_mean_time_resolution_pivot['merit_criminal'].astype(int)
bench_mean_time_resolution_pivot['other_civil'] = bench_mean_time_resolution_pivot['other_civil'].astype(int)
bench_mean_time_resolution_pivot['other_criminal'] = bench_mean_time_resolution_pivot['other_criminal'].astype(int)
bench_mean_time_resolution_pivot

#rename columns
bench_mean_time_resolution_pivot.columns = ['Bench', 'Merit Civil Timelines', 'Merit Criminal Timelines', 'Others Civil Timelines', 'Other Criminal Timelines']
bench_mean_time_resolution_pivot

,Bench,Merit Civil Timelines,Merit Criminal Timelines,Others Civil Timelines,Other Criminal Timelines
40,"Masiga, James Jesse",6334,0,0,0
72,"Odera, Teresia Achieng",4907,59,0,381
76,"Ogola, Eric Kennedy O.",4701,0,3145,0
59,"Nganga, Catherine",3509,0,3530,0
80,"Ongeri, Asenath",3088,0,623,0
...,...,...,...,...,...
49,"Musya, Musyoka William",0,0,0,337
26,"Karanja, Joseph R.",0,0,0,290
56,"Mwongo, Richard Mururu",0,0,0,265
52,"Mutende, Lilian Nambwire",0,0,0,89


In [26]:
#merge bench resolution time and bench resolution
bench_performance = pd.merge(resolution_bench_pivot, bench_mean_time_resolution_pivot, on='Bench', how='inner')
bench_performance

#save the combined file as a csv
bench_performance.to_csv(f'{path}/raw data/bench_performance.csv', index=False)